In [0]:
# Mount your google drive where you've saved your assignment folder
import torch
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# # For More RAM
# def function(l):
#     l.append([0]*500000000)
#     return l

# l=[]
# while True:
#   l=function(l)
  

In [0]:
cd /content/gdrive/My Drive/Project/transformers

/content/gdrive/My Drive/Project/transformers


In [0]:
pip install -r requirements.txt

     |████████████████████████████████| 675kB 2.8MB/s 
     |████████████████████████████████| 1.0MB 38.2MB/s 
     |████████████████████████████████| 860kB 57.3MB/s 
  Created wheel for regex: filename=regex-2019.12.9-cp36-cp36m-linux_x86_64.whl size=609176 sha256=db44066f1d7d0d68637038dec3c85aeda27b81c6392b553489ff132be7de2ea3
  Stored in directory: /root/.cache/pip/wheels/0d/fb/b3/a89169557229468c49ca64f6839418f22461f6ee0a74f342b1
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=391192e438a228a737ac97638032b7131a1cfadda1cc3ce21333f3e1a2d8b57e
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built regex sacremoses


In [0]:
!python setup.py build
!python setup.py install

running build
running build_py
running build_scripts
changing mode of build/scripts-3.6/transformers-cli from 600 to 755
running install
running bdist_egg
running egg_info
writing transformers.egg-info/PKG-INFO
writing dependency_links to transformers.egg-info/dependency_links.txt
writing entry points to transformers.egg-info/entry_points.txt
writing requirements to transformers.egg-info/requires.txt
writing top-level names to transformers.egg-info/top_level.txt
reading manifest template 'MANIFEST.in'
writing manifest file 'transformers.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/transformers
copying build/lib/transformers/modeling_camembert.py -> build/bdist.linux-x86_64/egg/transformers
copying build/lib/transformers/modeling_auto.py -> build/bdist.linux-x86_64/egg/transformers
copying build/lib/transformers/modeling_gpt2.py -> build/bdist

### CHECKING EMBEDDINGS

In [96]:
!pip install pytorch-pretrained-bert

In [0]:
trials = False

In [0]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
% matplotlib inline


if trials:
  # Load pre-trained model tokenizer (vocabulary)
  tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
  text = "Here is the sentence I want embeddings for."
  marked_text = "[CLS] " + text + " [SEP]"

  # Tokenize our sentence with the BERT tokenizer.
  tokenized_text = tokenizer.tokenize(marked_text)

  # Print out the tokens.
  print (tokenized_text)
  text = "As well as granting to protect the area of Rouen from Viking invasion, Rollo had to swear not to invade further Frankish lands himself,\
  accept baptism and conversion to the Roman Catholic faith of Christianity becoming Christian and swear fealty to King Charles III."

  # Add the special tokens.
  marked_text = "[CLS] " + text + " [SEP]"

  # Split the sentence into tokens.
  tokenized_text = tokenizer.tokenize(marked_text)

  # Map the token strings to their vocabulary indeces.
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

  # Display the words with their indeces.
  for tup in zip(tokenized_text, indexed_tokens):
      print('{:<12} {:>6,}'.format(tup[0], tup[1]))
  # Mark each of the 22 tokens as belonging to sentence "1".
  segments_ids = [1] * len(tokenized_text)

  print (segments_ids)
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])

  # Load pre-trained model (weights)
  model = BertModel.from_pretrained('bert-base-uncased')

  # Put the model in "evaluation" mode, meaning feed-forward operation.
  model.eval()

  with torch.no_grad():
      encoded_layers, _ = model(tokens_tensor, segments_tensors)
  for i, token_str in enumerate(tokenized_text):
    print (i, token_str)

  # create a new dimension in the tensor.
  token_embeddings = torch.stack(encoded_layers, dim=0)

  token_embeddings.size()
  # Remove dimension 1, the "batches".
  token_embeddings = torch.squeeze(token_embeddings, dim=1)

  token_embeddings.size()
  token_embeddings = token_embeddings.permute(1,0,2)

  token_embeddings.size()
  token_vecs_sum = []

  # `token_embeddings` is a [22 x 12 x 768] tensor.

  # For each token in the sentence...
  for token in token_embeddings:

      # `token` is a [12 x 768] tensor

      # Sum the vectors from the last four layers.
      sum_vec = torch.sum(token[-4:], dim=0)
      
      # Use `sum_vec` to represent `token`.
      token_vecs_sum.append(sum_vec)

  print ('Shape is: %d x %d' % (len(token_vecs_sum), len(token_vecs_sum[0])))
  print('First 5 vector values for each instance of "bank".')
  print('')
  print("bank vault   ", str(token_vecs_sum[6][:5]))
  print("bank robber  ", str(token_vecs_sum[10][:5]))
  print("river bank   ", str(token_vecs_sum[19][:5]))
  from scipy.spatial.distance import cosine

  # Calculate the cosine similarity between the word bank 
  # in "bank robber" vs "river bank" (different meanings).
  diff_bank = 1 - cosine(token_vecs_sum[10], token_vecs_sum[19])

  # Calculate the cosine similarity between the word bank
  # in "bank robber" vs "bank vault" (same meaning).
  same_bank = 1 - cosine(token_vecs_sum[10], token_vecs_sum[6])

  print('Vector similarity for  *similar*  meanings:  %.2f' % same_bank)
  print('Vector similarity for *different* meanings:  %.2f' % diff_bank)

100%|██████████| 231508/231508 [00:00<00:00, 5962377.61B/s]


In [0]:
import torch
torch.device('cuda:0')

device(type='cuda', index=0)

In [25]:
!python examples/run_squad.py \
  --model_type bert \
  --model_name_or_path bert-base-cased \
  --do_eval \
  --do_lower_case \
  --train_file train-v1.1.json \
  --predict_file dev-v1.1.json \
  --per_gpu_train_batch_size 12 \
  --learning_rate 3e-5 \
  --num_train_epochs 2.0 \
  --max_seq_length 384 \
  --doc_stride 128 \
  --output_dir /tmp/debug_squad/

12/10/2019 22:06:12 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
12/10/2019 22:06:12 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-config.json from cache at /root/.cache/torch/transformers/b945b69218e98b3e2c95acf911789741307dec43c698d35fad11c1ae28bda352.d7a3af18ce3a2ab7c0f48f04dc8daff45ed9a3ed333b9e9a79d012a0dedf87a6
12/10/2019 22:06:12 - INFO - transformers.configuration_utils -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "outpu

Niranjan Transcript from Recording: Take Bert that has been trained as a qa model. then ask learn photo. 
5:00
lower layer of BERT and add sentence prediction on this. So bert now knows about photosynthies. Attach qa again. bert qa about photosynthesis. 

100 question - photosysnthesis.


qa - ? -> Answ
bert - photo - ? -> see if better

In [26]:
cp /tmp/debug_squad 

debug_squad/  drivefs_ipc.0=  drivefs_ipc.0_shell=  tmpiayn3zlx/


In [27]:
!python examples/run_squad.py \
  --model_type bert \
  --model_name_or_path bert-base-cased \
  --do_eval \
  --do_lower_case \
  --train_file train-v1.1.json \
  --predict_file Norman.json \
  --per_gpu_train_batch_size 12 \
  --learning_rate 3e-5 \
  --num_train_epochs 2.0 \
  --max_seq_length 384 \
  --doc_stride 128 \
  --output_dir /tmp/debug_squad/

12/10/2019 22:30:19 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
12/10/2019 22:30:19 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-config.json from cache at /root/.cache/torch/transformers/b945b69218e98b3e2c95acf911789741307dec43c698d35fad11c1ae28bda352.d7a3af18ce3a2ab7c0f48f04dc8daff45ed9a3ed333b9e9a79d012a0dedf87a6
12/10/2019 22:30:19 - INFO - transformers.configuration_utils -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "outpu

In [0]:
!cp -rf /tmp/debug_squad/ '/content/gdrive/My Drive/Project/transformers/bert_base_cased_qa'

In [31]:
pwd

'/content/gdrive/My Drive/Project/transformers'

In [0]:
bert_base_qa_oxygen

In [36]:
!python examples/run_squad.py \
  --model_type bert \
  --model_name_or_path bert_base_qa_oxygen \
  --do_eval \
  --do_lower_case \
  --train_file train-v1.1.json \
  --predict_file dev-v1.1.json \
  --per_gpu_train_batch_size 12 \
  --learning_rate 3e-5 \
  --num_train_epochs 2.0 \
  --max_seq_length 384 \
  --doc_stride 128 \
  --output_dir bert_base_qa_oxygen

12/10/2019 23:04:45 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
12/10/2019 23:04:45 - INFO - transformers.configuration_utils -   loading configuration file bert_base_qa_oxygen/config.json
12/10/2019 23:04:45 - INFO - transformers.configuration_utils -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 28996
}

12/10/2019 23:04:45 - INFO - transformers.tokenization_utils -   Model name 'bert_base_qa

In [40]:
pwd

'/content/gdrive/My Drive/Project/transformers'

In [43]:
!python examples/run_squad.py \
  --model_type bert \
  --model_name_or_path bert_base_qa_oxygen \
  --do_eval \
  --do_lower_case \
  --train_file train-v1.1.json \
  --predict_file Oxygen.json \
  --per_gpu_train_batch_size 12 \
  --learning_rate 3e-5 \
  --num_train_epochs 2.0 \
  --max_seq_length 384 \
  --doc_stride 128 \
  --output_dir bert_base_qa_oxygen

12/10/2019 23:58:21 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
12/10/2019 23:58:21 - INFO - transformers.configuration_utils -   loading configuration file bert_base_qa_oxygen/config.json
12/10/2019 23:58:21 - INFO - transformers.configuration_utils -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 28996
}

12/10/2019 23:58:21 - INFO - transformers.tokenization_utils -   Model name 'bert_base_qa

In [45]:
!python examples/run_squad.py \
  --model_type bert \
  --model_name_or_path bert_base_qa_oxygen \
  --do_eval \
  --train_file train-v1.1.json \
  --predict_file Oxygen.json \
  --per_gpu_train_batch_size 12 \
  --learning_rate 3e-5 \
  --num_train_epochs 2.0 \
  --max_seq_length 384 \
  --doc_stride 128 \
  --output_dir bert_base_qa_oxygen

12/11/2019 00:01:49 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
12/11/2019 00:01:49 - INFO - transformers.configuration_utils -   loading configuration file bert_base_qa_oxygen/config.json
12/11/2019 00:01:49 - INFO - transformers.configuration_utils -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 28996
}

12/11/2019 00:01:49 - INFO - transformers.tokenization_utils -   Model name 'bert_base_qa

In [46]:
!python examples/run_squad.py \
  --model_type bert \
  --model_name_or_path bert_base_qa_oxygen \
  --do_eval \
  --train_file train-v1.1.json \
  --predict_file Oxygen.json \
  --per_gpu_train_batch_size 12 \
  --learning_rate 3e-5 \
  --num_train_epochs 2.0 \
  --max_seq_length 384 \
  --doc_stride 128 \
  --output_dir bert_base_qa_oxygen

12/11/2019 00:09:58 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
12/11/2019 00:09:58 - INFO - transformers.configuration_utils -   loading configuration file bert_base_qa_oxygen/config.json
12/11/2019 00:09:58 - INFO - transformers.configuration_utils -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 28996
}

12/11/2019 00:09:58 - INFO - transformers.tokenization_utils -   Model name 'bert_base_qa

In [49]:
!python examples/run_squad.py \
  --model_type bert \
  --model_name_or_path bert_base_qa_oxygen \
  --do_eval \
  --train_file train-v1.1.json \
  --predict_file Oxygen.json \
  --per_gpu_train_batch_size 12 \
  --learning_rate 3e-5 \
  --num_train_epochs 2.0 \
  --max_seq_length 384 \
  --doc_stride 128 \
  --output_dir bert_base_qa_oxygen

12/11/2019 00:16:41 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
12/11/2019 00:16:41 - INFO - transformers.configuration_utils -   loading configuration file bert_base_qa_oxygen/config.json
12/11/2019 00:16:41 - INFO - transformers.configuration_utils -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 28996
}

12/11/2019 00:16:41 - INFO - transformers.tokenization_utils -   Model name 'bert_base_qa

In [0]:
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
from pytorch_pretrained_bert import WEIGHTS_NAME, CONFIG_NAME
import os
# model = BertModel.from_pretrained('bert-base-uncased')
# model.eval()
output_dir = "bert_base_cased_qa"
# output_dir = "bert_base_uncased"
bert_based_cased_qa = BertModel.from_pretrained(output_dir)


In [0]:
qa_component = list(bert_based_cased_qa.encoder.children())

In [0]:
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
from pytorch_pretrained_bert import WEIGHTS_NAME, CONFIG_NAME
import os
# model = BertModel.from_pretrained('bert-base-uncased')
# model.eval()
output_dir = "bert_base_qa_oxygen"
# output_dir = "bert_base_uncased"
bert_base_qa_oxygen = BertModel.from_pretrained(output_dir)


In [0]:
embedding_component = list(bert_base_qa_oxygen.encoder.children())

In [0]:
list_=torch.nn.modules.container.ModuleList()
hybrid_component = qa_component[0][:8].extend(embedding_component[0][8:12])

In [0]:
bert_based_cased_qa_copy.encoder.children = hybrid_component

In [127]:
len(list(bert_based_cased_qa_copy.encoder.children())[0])

12

In [0]:
bert_based_cased_hybrid = bert_based_cased_qa

In [0]:
bert_based_cased_hybrid.encoder.children =  hybrid_component

In [0]:
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
from pytorch_pretrained_bert import WEIGHTS_NAME, CONFIG_NAME
import os

output_dir = "bert_based_cased_hybrid"
output_model_file = os.path.join(output_dir, WEIGHTS_NAME)
output_config_file = os.path.join(output_dir, CONFIG_NAME)
model_to_save = bert_based_cased_qa.module if hasattr(bert_based_cased_qa, 'module') else bert_based_cased_qa

# If we save using the predefined names, we can load using `from_pretrained`
output_model_file = os.path.join(output_dir, WEIGHTS_NAME)
output_config_file = os.path.join(output_dir, CONFIG_NAME)

torch.save(model_to_save.state_dict(), output_model_file)

In [138]:
ls

bert_base_cased_qa/
bert_based_cased_hybrid/
bert_base_qa_norman/
bert_base_qa_oxygen/
bert_base_uncased/
bert_base_uncased_oxygen/
bert_base_uncased_oxygen_8/
build/
cached_dev_bert-base-cased_384
cached_dev_bert_base_cased_qa_384
cached_dev_bert_base_qa_norman_384
cached_dev_bert_base_qa_oxygen_384
cached_dev_bert-large-uncased-whole-word-masking-finetuned-squad_384
cached_train_bert-base-cased_384
CONTRIBUTING.md
data/
deploy_multi_version_doc.sh
dev-v1.1.json
dist/
docker/
docs/
evaluate-v1.1.py
examples/
hubconf.py
LICENSE
MANIFEST.in
Norman.json
notebooks/
output/
output_2/
output_normans/
output_normans_qa/
output_normans_qa_bert_base/
output_normans_qa_bert_base_large/
Oxygen.json
Oxygen.txt
README.md
requirements-dev.txt
requirements.txt
runs/
setup.py
templates/
train-v1.1.json
transformers/
transformers-cli
transformers.egg-info/
utils/
valohai.yaml
working/
working_2/


In [0]:
model_to_save.config.to_json_file(output_config_file)

In [153]:
!python examples/run_squad.py \
  --model_type bert \
  --model_name_or_path bert_base_uncased_oxygen \
  --do_eval \
  --train_file train-v1.1.json \
  --predict_file Oxygen.json \
  --per_gpu_train_batch_size 12 \
  --learning_rate 3e-5 \
  --num_train_epochs 2.0 \
  --max_seq_length 384 \
  --doc_stride 128 \
  --output_dir bert_base_uncased_oxygen

12/11/2019 01:10:28 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
12/11/2019 01:10:28 - INFO - transformers.configuration_utils -   loading configuration file bert_base_uncased_oxygen/config.json
12/11/2019 01:10:28 - INFO - transformers.configuration_utils -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 28996
}

12/11/2019 01:10:28 - INFO - transformers.tokenization_utils -   Model name 'bert_ba